# Phase 2 – Function Calling & Tooling (LangChain + OpenAI Tools)

Notebook khởi tạo. Các cell nội dung chi tiết sẽ được thêm ở bước tiếp theo.

In [ ]:
print('Notebook skeleton created successfully.')


## 0. Yêu cầu & Chuẩn bị
- Python 3.10+
- OpenAI API Key (`OPENAI_API_KEY` trong env hoặc `.env`)
- Thư viện: `langchain`, `langchain-openai`, `tenacity`, `python-dotenv`, `rich`, `langgraph` (bonus)


In [ ]:

# (Chạy ở máy của bạn) Cài đặt/thăng cấp thư viện
# !pip install -U langchain langchain-openai tenacity python-dotenv rich langgraph



## 1. Khởi tạo LLM & Logging


In [1]:

import os, json, time, logging
from typing import Any, Dict, List, Optional
from dotenv import load_dotenv
load_dotenv()

logger = logging.getLogger("phase2_tools")
logger.setLevel(logging.INFO)
if not logger.handlers:
    ch = logging.StreamHandler()
    ch.setLevel(logging.INFO)
    ch.setFormatter(logging.Formatter("[%(asctime)s] %(levelname)s - %(message)s"))
    logger.addHandler(ch)

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    logger.warning("⚠️ OPENAI_API_KEY chưa có.")
else:
    logger.info("✅ Đã đọc OPENAI_API_KEY.")


[2025-11-05 18:35:31,670] INFO - ✅ Đã đọc OPENAI_API_KEY.


In [2]:

from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage, ToolMessage
from langchain_core.tools import tool, StructuredTool
from tenacity import retry, wait_exponential, stop_after_attempt

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)
logger.info("✅ Khởi tạo ChatOpenAI.")


[2025-11-05 18:35:40,185] INFO - ✅ Khởi tạo ChatOpenAI.



## 2. Định nghĩa Tool (Pydantic + StructuredTool)
- `calculator(expression: str)`
- `unit_convert(value, from_unit, to_unit)`
- `fake_weather(city)`


In [3]:

import ast, operator as op
from pydantic import BaseModel, Field, validator

ALLOWED_OPS = {
    ast.Add: op.add, ast.Sub: op.sub, ast.Mult: op.mul, ast.Div: op.truediv,
    ast.Pow: op.pow, ast.USub: op.neg, ast.Mod: op.mod
}

def _eval_expr(node):
    if isinstance(node, ast.Num): return node.n
    if hasattr(ast, "Constant") and isinstance(node, ast.Constant): return node.value
    if isinstance(node, ast.BinOp):
        if type(node.op) not in ALLOWED_OPS:
            raise ValueError(f"Operator {type(node.op).__name__} không được phép.")
        return ALLOWED_OPS[type(node.op)](_eval_expr(node.left), _eval_expr(node.right))
    if isinstance(node, ast.UnaryOp):
        if type(node.op) not in ALLOWED_OPS:
            raise ValueError(f"Unary {type(node.op).__name__} không được phép.")
        return ALLOWED_OPS[type(node.op)](_eval_expr(node.operand))
    raise ValueError("Biểu thức không hợp lệ.")

def safe_eval(expr: str) -> float:
    try:
        tree = ast.parse(expr, mode="eval")
        return float(_eval_expr(tree.body))
    except Exception as e:
        raise ValueError(f"Không thể tính biểu thức: {e}") from e

class CalcInput(BaseModel):
    expression: str = Field(..., description="Biểu thức số học, ví dụ: '12*(3+5)-4^2'.")

@tool("calculator", args_schema=CalcInput)
def calculator(expression: str) -> float:
    """Tính toán biểu thức số học an toàn và trả về kết quả float."""
    return safe_eval(expression)

class UnitConvertInput(BaseModel):
    value: float = Field(...)
    from_unit: str = Field(...)
    to_unit: str = Field(...)

    @validator("from_unit", "to_unit")
    def normalize(cls, v):
        return v.strip().lower()

SUPPORTED_UNITS = {"m": 1.0, "km": 1000.0, "cm": 0.01}

@tool("unit_convert", args_schema=UnitConvertInput)
def unit_convert(value: float, from_unit: str, to_unit: str) -> float:
    """Chuyển đổi m/km/cm. Raise lỗi nếu đơn vị không hỗ trợ."""
    if from_unit not in SUPPORTED_UNITS or to_unit not in SUPPORTED_UNITS:
        raise ValueError(f"Đơn vị không hỗ trợ: {from_unit}->{to_unit}. Hỗ trợ: {sorted(SUPPORTED_UNITS)}")
    meters = value * SUPPORTED_UNITS[from_unit]
    return meters / SUPPORTED_UNITS[to_unit]

class WeatherInput(BaseModel):
    city: str = Field(..., description="Tên thành phố.")

@tool("fake_weather", args_schema=WeatherInput)
def fake_weather(city: str) -> dict:
    """Trả thời tiết giả lập (không gọi web)."""
    sample = {
        "Hanoi": {"temp_c": 29.0, "status": "Cloudy"},
        "Ho Chi Minh City": {"temp_c": 32.0, "status": "Sunny"},
        "Da Nang": {"temp_c": 30.0, "status": "Windy"},
    }
    key = city.strip()
    return sample.get(key, {"temp_c": 28.0, "status": "Partly cloudy", "note": "city not in sample"})

TOOLS = [calculator, unit_convert, fake_weather]
logger.info("✅ TOOLS: %s", [t.name for t in TOOLS])


C:\Users\hung.hm\AppData\Local\Temp\ipykernel_20188\2901122037.py:42: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  @validator("from_unit", "to_unit")
[2025-11-05 18:37:58,418] INFO - ✅ TOOLS: ['calculator', 'unit_convert', 'fake_weather']



## 3. bind_tools + Executor loop
1) Gọi LLM (đã bind tools)  
2) Nếu có `tool_calls` → thực thi tool → trả `ToolMessage`  
3) Lặp cho đến khi không còn `tool_calls`


In [4]:

@retry(wait=wait_exponential(multiplier=1, min=1, max=8), stop=stop_after_attempt(3))
def call_llm(messages, tools=None):
    model = llm if tools is None else llm.bind_tools(tools)
    return model.invoke(messages)

def execute_tool_call(tool_call: dict) -> str:
    name = tool_call.get("name")
    args = tool_call.get("args", {}) or {}
    tmap = {t.name: t for t in TOOLS}
    start = time.time()
    try:
        if name not in tmap:
            raise ValueError(f"Tool '{name}' không tồn tại.")
        result = tmap[name].invoke(args)
        payload = {"ok": True, "result": result}
    except Exception as e:
        payload = {"ok": False, "error": str(e), "tool": name, "args": args}
        logger.exception("❌ Tool error: %s", payload)
    finally:
        dur = (time.time() - start) * 1000
        logger.info("🔧 %s(%s) -> %.1fms", name, args, dur)
    return json.dumps(payload, ensure_ascii=False)

def run_with_tools(user_prompt: str, tools, system_prompt: str=None, max_steps: int=3):
    messages = []
    if system_prompt:
        messages.append(SystemMessage(content=system_prompt))
    messages.append(HumanMessage(content=user_prompt))

    for _ in range(max_steps):
        ai = call_llm(messages, tools)
        messages.append(ai)
        if not getattr(ai, "tool_calls", None):
            return ai.content, messages
        for tc in ai.tool_calls:
            payload = execute_tool_call(tc)
            messages.append(ToolMessage(content=payload, tool_call_id=tc["id"]))
    return "⚠️ Quá số bước cho phép.", messages


### Demo 1 – Calculator

In [5]:

final_answer, trace = run_with_tools(
    user_prompt="Hãy tính: 12 * (3 + 5) - 4^2. Trả lời 1 con số.",
    tools=TOOLS,
    system_prompt="Bạn là trợ lý toán học. Chỉ trả lời kết quả cuối cùng.",
    max_steps=3,
)
print("Final:", final_answer)


C:\Users\hung.hm\AppData\Local\Temp\ipykernel_20188\2901122037.py:10: DeprecationWarning: ast.Num is deprecated and will be removed in Python 3.14; use ast.Constant instead
  if isinstance(node, ast.Num): return node.n
[2025-11-05 18:38:44,550] ERROR - ❌ Tool error: {'ok': False, 'error': 'Không thể tính biểu thức: Operator BitXor không được phép.', 'tool': 'calculator', 'args': {'expression': '12*(3+5)-4^2'}}
Traceback (most recent call last):
  File "C:\Users\hung.hm\AppData\Local\Temp\ipykernel_20188\2901122037.py", line 25, in safe_eval
    return float(_eval_expr(tree.body))
                 ^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\hung.hm\AppData\Local\Temp\ipykernel_20188\2901122037.py", line 14, in _eval_expr
    raise ValueError(f"Operator {type(node.op).__name__} không được phép.")
ValueError: Operator BitXor không được phép.

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\Users\hung.hm\AppData\Local\Temp\ipyk

Final: 80.0


### Demo 2 – Validate & tự sửa lỗi tool-call

In [6]:

final_answer, trace = run_with_tools(
    user_prompt="Đổi 1.5 mile sang km. Nếu tool không hỗ trợ, hãy tự chuyển đổi qua m rồi sang km.",
    tools=TOOLS,
    system_prompt="Bạn là trợ lý kỹ thuật.",
    max_steps=4,
)
print("Final:", final_answer)


[2025-11-05 18:39:35,919] ERROR - ❌ Tool error: {'ok': False, 'error': "Đơn vị không hỗ trợ: mile->m. Hỗ trợ: ['cm', 'km', 'm']", 'tool': 'unit_convert', 'args': {'value': 1.5, 'from_unit': 'mile', 'to_unit': 'm'}}
Traceback (most recent call last):
  File "C:\Users\hung.hm\AppData\Local\Temp\ipykernel_20188\3221833393.py", line 14, in execute_tool_call
    result = tmap[name].invoke(args)
             ^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\stockvip\academy\AI Agent\venv\Lib\site-packages\langchain_core\tools\base.py", line 598, in invoke
    return self.run(tool_input, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\stockvip\academy\AI Agent\venv\Lib\site-packages\langchain_core\tools\base.py", line 904, in run
    raise error_to_raise
  File "d:\stockvip\academy\AI Agent\venv\Lib\site-packages\langchain_core\tools\base.py", line 873, in run
    response = context.run(self._run, *tool_args, **tool_kwargs)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  Fi

Final: Để chuyển đổi 1.5 mile sang km, ta có thể sử dụng công thức chuyển đổi:

1 mile = 1609.34 m.

Vậy 1.5 mile = 1.5 * 1609.34 m = 2414.01 m.

Sau đó, chuyển đổi từ mét sang km:

2414.01 m = 2414.01 / 1000 = 2.41401 km.

Vậy 1.5 mile tương đương khoảng 2.41 km.


### Demo 3 – JSON tool output

In [ ]:

final_answer, trace = run_with_tools(
    user_prompt="Thời tiết hôm nay ở Ho Chi Minh City?",
    tools=TOOLS,
    system_prompt="Bạn là trợ lý thời tiết, tóm tắt ngắn gọn.",
    max_steps=3,
)
print("Final:", final_answer)


## 4. Kiểm tra trace

In [ ]:

from pprint import pprint
for i, msg in enumerate(trace, 1):
    role = type(msg).__name__
    print(f"\n--- [{i}] {role} ---")
    if isinstance(msg, AIMessage):
        print("tool_calls:", msg.tool_calls)
    print(msg.content if isinstance(msg.content, str) else msg.content)



## 5. StructuredTool (tạo thủ công)


In [ ]:

from pydantic import BaseModel, Field

class MultiplyInput(BaseModel):
    a: float = Field(...)
    b: float = Field(...)

def _multiply(a: float, b: float) -> float:
    return a * b

multiply_tool = StructuredTool.from_function(
    name="multiply",
    description="Nhân 2 số a và b.",
    func=_multiply,
    args_schema=MultiplyInput,
)

EXTRA_TOOLS = TOOLS + [multiply_tool]

final_answer, _ = run_with_tools(
    user_prompt="Tính 3.5 * 7.2 bằng tool multiply.",
    tools=EXTRA_TOOLS,
    system_prompt="Bạn là trợ lý toán học.",
    max_steps=3,
)
print("Final:", final_answer)



## 6. Bonus – LangGraph với ToolNode


In [ ]:

from typing import TypedDict, List, Any
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolNode
from langchain_core.messages import AIMessage

class GraphState(TypedDict):
    messages: List[Any]

def llm_node(state: GraphState):
    msgs = state["messages"]
    ai = llm.bind_tools(TOOLS).invoke(msgs)
    return {"messages": msgs + [ai]}

tool_node = ToolNode(TOOLS)

def should_call_tool(state: GraphState):
    msgs = state["messages"]
    last = msgs[-1]
    if isinstance(last, AIMessage) and getattr(last, "tool_calls", None):
        return "tools"
    return END

g = StateGraph(GraphState)
g.add_node("llm", llm_node)
g.add_node("tools", tool_node)
g.set_entry_point("llm")
g.add_conditional_edges("llm", should_call_tool, {"tools": "tools", END: END})
g.add_edge("tools", "llm")

app = g.compile()

state = {"messages": [HumanMessage(content="Tính (100 - 64) / 2 và trả một số duy nhất.")]}  # noqa
out = app.invoke(state)
final_msg = out["messages"][-1]
print("Final:", final_msg.content)



## 7. Best Practices & Bài tập
**Best Practices**
- Retry LLM bằng `tenacity`.
- Validate input tool bằng Pydantic.
- Raise lỗi có ý nghĩa → cho LLM tự sửa.
- Log đầy đủ: tool, args, duration, ok/error.
- Giới hạn vòng lặp tool-call (`max_steps`).

**Bài tập**
1) Viết tool `ticker_info(symbol)` (mock) + validate `symbol` (`A-Z`, 1–5 ký tự).  
2) Chuỗi tool: `ticker_info` → `unit_convert` (nếu có field cần đổi đơn vị).  
3) Thêm `@retry` riêng cho tool có I/O (giả lập lỗi 429).  
4) Logging đẹp hơn bằng `rich.logging`.  
5) LangGraph: Thêm node `finalize_llm` để tóm tắt đầu ra.
